In [ ]:
!pip install holidays -q
!pip install workalendar -q
!pip install fasttext -q

In [24]:
%%capture
import catboost
import pandas as pd
import numpy as np
import fasttext
import fasttext.util
#fasttext.util.download_model("ru")
#fasttext.util.reduce_model(ft_model, 100)
ft_model = fasttext.load_model("cc.ru.300.bin")

In [25]:
file_name = r"C:\Users\True7\Downloads\TenderHack_Москва_train_data.xlsx" 
columns = ['id', 'status', 'name', 'okpd2','kpgz','region','nmck','price','date','participans','bet','inn']

spgz = "D:\Projects\lct_8\Исходные данные\Исходные данные\Список СПГЗ 27_5_2022.xlsx"
df_spgz = pd.read_excel(spgz)
df_spgz = df_spgz['КПГЗ']

df = pd.read_excel(file_name, header=None, names=columns, skiprows=1)
df['okpd2'] = df['okpd2'].fillna(0)
prod_df = df
prod_df = prod_df.drop('id', axis='columns')
prod_df.dtypes

status          object
name            object
okpd2           object
kpgz            object
region          object
nmck           float64
price          float64
date            object
participans      int64
bet              int64
inn             object
dtype: object

In [26]:
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246762 entries, 0 to 246761
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   status       246762 non-null  object 
 1   name         246762 non-null  object 
 2   okpd2        246762 non-null  object 
 3   kpgz         224212 non-null  object 
 4   region       246762 non-null  object 
 5   nmck         246762 non-null  float64
 6   price        246762 non-null  float64
 7   date         246762 non-null  object 
 8   participans  246762 non-null  int64  
 9   bet          246762 non-null  int64  
 10  inn          246762 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 20.7+ MB


In [27]:
prod_df.head()

,status,name,okpd2,kpgz,region,nmck,price,date,participans,bet,inn
0,Завершена,СТУЛЬЯ УЧЕНИЧЕСКИЕ,0,01.06.01.03.01;01.06.01.03.01,Москва,596790.0,593806.05,2021-06-30 11:20:05.720,1,1,d8912494ba2edd3bfeba55206a78a0f5
1,Не состоялась,МЕДИЦИНСКИЕ РАСХОДНЫЕ МАТЕРИАЛЫ,0,01.02.10.50.33;01.02.10.43.05.01,Москва,4964.9,0.00,2022-10-04 09:16:04.770,0,0,237a5c57a66f02f8deb152e21f33863a
2,Завершена,Видеокарта Palit PCI-E,0,01.13.17.08,Москва,462000.0,311850.00,2021-07-01 13:23:09.177,6,65,040f1570117a744d529f4675f827a50f
3,Завершена,"ТОВАРЫ ИНФОРМАЦИОННО-ТЕХНОЛОГИЧЕСКИЕ, СРЕДСТВА...",0,01.13.11.03.02;01.13.04.04.06.05;01.10.04.04.0...,Москва,505555.0,460042.90,2021-03-16 10:42:20.810,6,18,c00c03dca0274fe43fc34e974434a927
4,Завершена,Пинцет острый,0,01.02.10.06.48.04,Москва,2000.0,1980.00,2022-05-26 11:04:42.597,2,2,cc997efa7a6742b6119a3c253a084e80


In [28]:
"""
Завершена
Не состоялась
Снята с публикации
Активна"""

prod_df["status"].value_counts()
cleanup_nums = {"status": {"Завершена": 1, "Не состоялась": 0, 'Снята с публикации': 0, 'Активна': -1}}
prod_df = prod_df.replace(cleanup_nums)

prod_df.head()

print(prod_df["status"].unique())

[ 1  0 -1]


In [29]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for cat_var in ['inn']:
    prod_df[cat_var] = le.fit_transform(prod_df[cat_var])  
    
print(prod_df["inn"].unique())
print(prod_df["inn"].max())

[2808  443   56 ... 2689 1099  492]
3334


In [31]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for cat_var in ['region']:
    prod_df[cat_var] = le.fit_transform(prod_df[cat_var])  
    
print(prod_df["region"].unique())
print(prod_df["region"].max())

[2 4 5 7 9 6 1 0 3 8]
9


In [33]:
def get_names_embeddings(kpgzs: str, kpgz_book: pd.DataFrame, ft_model) -> str:
    kpgzs = kpgzs.strip()
    if(not kpgzs):
        return None
    
    result = []
    for kpgz in kpgzs.split(";"):
        result.append(" ".join(prod_df[prod_df.kpgz.str.split().str[0] == kpgz].iloc[0].kpgz.split()[1:]))
    return ft_model.get_sentence_vector(" ".join(result))
        
kpgzs = prod_df["kpgz"].map(lambda x:get_names_embeddings(x, kpgz_book=prod_df, ft_model=ft_model))
kpgzs

# prod_df = prod_df.drop('name', axis='columns')
# prod_df = prod_df.drop('okpd2', axis='columns')
# prod_df = prod_df.drop('kpgz', axis='columns')

AttributeError: 'float' object has no attribute 'strip'

In [ ]:
prod_df['date'] = pd.to_datetime(prod_df.date, format='%Y-%m-%d %H:%M:%S')
import holidays
from workalendar.europe import Russia

range_of_dates = pd.date_range("2022-01-01", "2023-12-31")
holidays = holidays.Russia(years=[2022, 2023]).items()
hdays = []
for day in holidays:
    hdays.append(day[0])

def season_from_date(date):
    year = str(date.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
               'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
    if date in seasons['spring']:
        return '1'
    if date in seasons['summer']:
        return '2'
    if date in seasons['autumn']:
        return '3'
    else:
        return '4'

df_dt = pd.DataFrame()
df_dt['date_year'] = prod_df['date'].dt.year
df_dt['date_month'] = prod_df['date'].dt.month
df_dt['date_week'] = prod_df['date'].dt.isocalendar().week 
df_dt["date_weekend"] = df_dt["date_week"] >= 5
df_dt["date_weekend"] = df_dt["date_weekend"].astype(int)
df_dt['date_day'] = prod_df['date'].dt.day
df_dt['date_hour'] = prod_df['date'].dt.hour
df_dt['date_minute'] = prod_df['date'].dt.minute
df_dt['date_dayofweek'] = prod_df['date'].dt.dayofweek
df_dt['date_quarter'] = prod_df['date'].dt.quarter
#df_dt["delta_to_holiday"] = df['date'].isin(hdays)
df_dt["is_holiday"] = prod_df['date'].isin(hdays)
df_dt["is_holiday"] = df_dt["is_holiday"].astype(int)
df_dt['date_season'] = prod_df.date.map(season_from_date)
prod_df = pd.concat([prod_df, df_dt], axis=1)
prod_df = prod_df.drop('date', axis='columns')
prod_df

# Set target and drop from df

In [ ]:
X = prod_df.drop("status", axis=1)
y = prod_df["status"]

# Cat features

In [ ]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    #loss_function='CrossEntropy'
)


clf.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_val, y_val), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

In [ ]:
# import re


# for idx, row in enumerate(df_spgz):
#     kpgz = re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', row).group()
#     print(kpgz)
#     if idx > 10:
#         break
    

In [ ]:
# """
# 1, 2, 3, 4, 5, 6 and 8 January — New Year Holidays;

# 7 January — Christmas Day;

# 23 February — Defender of the Fatherland Day;

# 8 March — International Women’s Day;

# 1 May — Spring and Labour Holiday;

# 9 May — Victory Day;

# 12 June — Day of Russia;

# 4 November — National Unity Day.
# """

# class RusHolidaysBusinessCalendar(AbstractHolidayCalendar):
#    rules = [
#      Holiday('New Year', month=12, day=31, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=1, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=2, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=3, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=4, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=5, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=6, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=8, observance=sunday_to_monday),
#      Holiday('8 March', month=3, day=8, observance=sunday_to_monday),
#      Holiday('1 May', month=5, day=1, observance=sunday_to_monday),
#      Holiday('9 May', month=5, day=9),
#      Holiday('12 June', month=6, day=12, observance=sunday_to_monday),
#      Holiday('4 November', month=5, day=1, observance=sunday_to_monday),
#      Holiday('11 November', month=11, day=11, observance=sunday_to_monday),
#    ]

# rus_BD = CustomBusinessDay(calendar=RusHolidaysBusinessCalendar())
# s = pd.date_range('2022-01-01', end='2022-12-31', freq=rus_BD)
# df = df.resample(rule=CustomBusinessMonthEnd(holidays=rus_BD)).last()
# df = pd.DataFrame(s, columns=['date'])
# print(df.tail())